In [1]:
import tensorflow as tf
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json

from tqdm import tqdm

# Load Data set

In [2]:
DATA_IN_PATH = './preprocessed_data/'
DATA_OUT_PATH = './data_out/'
INPUT_TRAIN_DATA = 'nsmc_train_input.npy'
LABEL_TRAIN_DATA = 'nsmc_train_label.npy'
DATA_CONFIGS = 'data_configs.json'

In [3]:
train_input = np.load(open(DATA_IN_PATH + INPUT_TRAIN_DATA, 'rb'))
train_input = pad_sequences(train_input, maxlen=train_input.shape[1])

In [4]:
train_label = np.load(open(DATA_IN_PATH + LABEL_TRAIN_DATA, 'rb'))

In [5]:
prepro_configs = json.load(open(DATA_IN_PATH + DATA_CONFIGS, 'r'))

# Define Hyper-parameters

In [6]:
model_name = 'cnn_classifier_kr'
BATCH_SIZE = 512
NUM_EPOCHS = 2
VALID_SPLIT = 0.1
MAX_LEN = train_input.shape[1]

kargs = {'model_name' : model_name,
         'vocab_size' : prepro_configs['vocab_size'],
         'embedding_size' : 128,
         'num_filters' : 100,
         'dropout_rate' : 0.5,
         'hidden_dimension' : 250,
         'output_dimension' : 1}

In [7]:
class CNNClassifier(tf.keras.Model):
    def __init__(self, **kargs):
        super(CNNClassifier, self).__init__(name=kargs['model_name'])
        self.embedding = tf.keras.layers.Embedding(input_dim=kargs['vocab_size'], output_dim=kargs['embedding_size'])
        self.conv_list = [tf.keras.layers.Conv1D(filters=kargs['num_filters'],
                                                 kernel_size=kernel_size,
                                                 padding='valid',
                                                 activation=tf.keras.activations.relu,
                                                 kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.)) for kernel_size in [3, 4, 5]]
        self.pooling = tf.keras.layers.GlobalMaxPooling1D()
        self.dropout = tf.keras.layers.Dropout(kargs['dropout_rate'])
        self.fc1 = tf.keras.layers.Dense(units=kargs['hidden_dimension'], activation=tf.keras.activations.relu, kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
        self.fc2 = tf.keras.layers.Dense(units=kargs['output_dimension'], activation=tf.keras.activations.sigmoid, kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))

    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = tf.concat([self.pooling(conv(x)) for conv in self.conv_list], axis=-1)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

In [8]:
model = CNNClassifier(**kargs)

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')])

In [10]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=2, mode='max')

In [11]:
checkpoint_path = DATA_OUT_PATH+model_name+'/checkpoint.weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

In [12]:
if os.path.exists(checkpoint_dir):
    print('{} -- Folder already exists \n'.format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print('{} -- Folder create complete \n'.format(checkpoint_dir))

./data_out/cnn_classifier_kr -- Folder already exists 



In [13]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True
)

In [14]:
history = model.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, validation_split=VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

Epoch 1/2
263/264 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.7365 - loss: 0.5074
Epoch 1: val_accuracy improved from -inf to 0.82700, saving model to ./data_out/cnn_classifier_kr/checkpoint.weights.h5
264/264 ━━━━━━━━━━━━━━━━━━━━ 9s 31ms/step - accuracy: 0.7369 - loss: 0.5069 - val_accuracy: 0.8270 - val_loss: 0.3803
Epoch 2/2
263/264 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8784 - loss: 0.2932
Epoch 2: val_accuracy improved from 0.82700 to 0.82760, saving model to ./data_out/cnn_classifier_kr/checkpoint.weights.h5
264/264 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.8784 - loss: 0.2932 - val_accuracy: 0.8276 - val_loss: 0.3877


# Evaluate TEST data

In [15]:
INPUT_TEST_DATA = 'nsmc_test_input.npy'
LABEL_TEST_DATA = 'nsmc_test_label.npy'
SAVE_FILE_NM = 'checkpoint.weights.h5'

In [16]:
test_input = np.load(open(DATA_IN_PATH + INPUT_TEST_DATA, 'rb'))
test_input = pad_sequences(test_input, maxlen=test_input.shape[1])

In [17]:
test_label_data = np.load(open(DATA_IN_PATH + LABEL_TEST_DATA, 'rb'))

In [18]:
model.load_weights(os.path.join(DATA_OUT_PATH, model_name, SAVE_FILE_NM))

In [19]:
model.evaluate(test_input, test_label_data)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 864us/step - accuracy: 0.8260 - loss: 0.3912


[0.3932075798511505, 0.8246999979019165]